### Bloomberg data

In [5]:
from newsapi import NewsApiClient
import json
from math import ceil
import logging
import datetime
import pymysql
import pandas as pd
import queue
import gensim

D:\Tools\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
newsapi = NewsApiClient(api_key='b5396dad3ff14d0b8ef6307d4daee12d')

NameError: name 'NewsApiClient' is not defined

In [134]:
newsapi = NewsApiClient(api_key='4cb82bcecd6c4c9586496b9851c102a4')

In [66]:
newsapi = NewsApiClient(api_key='49c22402bc7d4a669d0e393e556f45bc')

In [61]:
data = newsapi.get_everything(q = '', 
                               sources = 'bloomberg',
                               from_param = '2017-12-17',
                               to = '2018-01-16',
                               #category = 'market',
                               language = 'en',
                               #country = 'us',
                               page_size = 100,
                               page = 33
                               )

ValueError: from_param should be in the format of YYYY-MM-DD

In [58]:
len(data['articles'])

33

In [37]:
for news in data['articles']:
    print(news['title'])

Sugar Bears Come Out to Play as Surplus Swells on Asia Harvests
Trump Moves Stormy Daniels Lawsuit to Federal Court
Saudi Crown Prince Plans Meetings With Apple, Google
Dow's Outgoing Chairman Got $65.7 Million in Pay for Last Year
Tim Cook Meets Treasury Secretary Mnuchin at Apple Headquarters - Bloomberg
Hollywood Academy President Faces Sexual-Harassment Allegations
Kelly Tells White House Staff Their Jobs Are Safe
Brazil Mourns the Murder of a Rare Hero - Bloomberg
Nike Executive Departures Accompany Reports of Demeaning Conduct - Bloomberg
iHeart Creditors Are Said to Be Open to Bids Including Liberty's
U.K. Trade Chief Says Trump Tariffs Upsetting Allies, Not China
Toys `R' Us Is Shuttering Stores, and Few Will Want to Fill Them
The Other Job Mitt Romney Is Running For
U.S. Weighs Nafta Telecom Proposal to End AT&T Impasse
New Yorkers Brace for Rough Commute as Fire Stalls LIRR Lines
U.K. Points Finger at Putin Over Nerve Agent Attack
Your Evening Briefing
Irish PM Says 'Think We

In [60]:
data['articles'][0]

{'author': 'James Mayger',
 'description': "Bloomberg BOJ's Massive Bond Holdings Pace Seen Expanding Even More Slowly in 2018 Bloomberg The pace at which the Bank of Japan is expanding its massive hoard of bonds will continue to slow in 2018, according to the majority of economists surveyed by Bloombe…",
 'publishedAt': '2017-12-17T20:44:46Z',
 'source': {'id': 'bloomberg', 'name': 'Bloomberg'},
 'title': "BOJ's Massive Bond Holdings Pace Seen Expanding Even More Slowly in 2018 - Bloomberg",
 'url': 'https://www.bloomberg.com/news/articles/2017-12-17/boj-s-holdings-of-jgbs-is-seen-expanding-even-more-slowly-in-18',
 'urlToImage': 'https://assets.bwbx.io/images/users/iqjWHBFdfxIU/inNRmciQljqk/v0/1200x801.jpg'}

In [39]:
with open('page%d'%1, 'at') as f:
    f.write(json.dumps(data))

In [81]:
with open('headers.2018.03.json', 'rt') as f:
    data = json.loads(f.read())

In [84]:
[news['title'] for news in data]

['House GOP Looks to Sweeten Spending Deal for Conservatives',
 'Reidl Sees a 50/50 Chance of U.S. Govt. Shutdown',
 'Google Tightens YouTube Rules to Clean It Up for Advertisers - Bloomberg',
 'Venezuela Plans Discounts, Tax Deals to Bolster Cryptocurrency',
 'The Case Against College',
 'Fox Is Trying to Win the Rights to Thursday NFL Games',
 'BitConnect Closes Exchange as States Warn of Unregulated Sales - Bloomberg',
 'Ford Warns Profit Will Drop This Year',
 "Trump International Is Bullying Hotel's Owners, Lawsuit Claims",
 "Global Businesses Scramble to Align With China's World View",
 'Five Things You Need to Know to Start Your Day',
 'Boeing Barges Into Aircraft-Seat Business in Threat to Suppliers - Bloomberg',
 'House Panel Subpoenas Steve Bannon',
 "A Steelmaker's Plan to Win Back the Car of the Future",
 "L'Oral Is Growing Chinese Skin to Test Products",
 'India May Be Facing Its Own Subprime Problem',
 "Trump's CFPB to Rewrite Rule That Cracked Down on Payday Lenders",
 '

In [54]:
data['totalResults']

3233

### Get News Headers from 20170701 to 20180716

In [67]:
def SaveNewsHeaders(from_date, to_date, json_name, news_source = 'bloomberg'):
    # Get total number of news
    data = newsapi.get_everything(q = '', 
                               sources = news_source,
                               from_param = from_date,
                               to = to_date,
                               #category = 'market',
                               language = 'en',
                               #country = 'us',
                               page_size = 1,
                               page = 1
                               )
    num_news = data['totalResults']
    num_pages = ceil(num_news/100)
    if num_pages >= 100:
        logger.error('More than 99 pages when querying data from %s to %s, in page %d/%d'%(from_date, to_date, page, num_pages))
        num_pages = 99
    headers = []
    for page in range(1, num_pages + 1):
        print('current progress: %d/%d, header length: %d'%(page, num_pages, len(headers)))
        data = newsapi.get_everything(q = '', 
                               sources = news_source,
                               from_param = from_date,
                               to = to_date,
                               #category = 'market',
                               language = 'en',
                               #country = 'us',
                               page_size = 100,
                               page = page
                               )
        headers += data['articles']
        if data['status'] != 'ok':
            logger.error('Status is not "ok" when querying data from %s to %s, in page %d/%d'%(from_date, to_date, page, num_pages))
            continue
    with open(json_name, 'at') as f:
        f.write(json.dumps(headers))
    return headers

In [70]:
dataf = SaveNewsHeaders('2018-03-01', '2018-03-31', 'financial_post.headers.2018.03.json', news_source = 'financial-post')

current progress: 1/5, header length: 0
current progress: 2/5, header length: 100
current progress: 3/5, header length: 200
current progress: 4/5, header length: 300
current progress: 5/5, header length: 400


In [75]:
dataf[3]

{'author': 'Kevin Carmichael',
 'description': 'Kevin Carmichael: It’s a puzzle how the federal government, Ontario and Quebec could present financial plans that would do little, if anything, to offset what Trump is doing to business sentiment',
 'publishedAt': '2018-03-30T10:30:34Z',
 'source': {'id': 'financial-post', 'name': 'Financial Post'},
 'title': 'Finance ministers put emphasis where it shouldn’t be — on social issues',
 'url': 'http://business.financialpost.com/news/economy/finance-ministers-put-emphasis-where-it-shouldnt-be',
 'urlToImage': 'http://financialpostcom.files.wordpress.com/2018/03/ontario-budget-1.png'}

In [70]:
data['status']

'ok'

In [20]:
logger = logging.getLogger('main_logger')
logger.setLevel(logging.INFO)
logger.addHandler(logging.FileHandler(filename = 'getnewsheaders.log', mode = 'a'))
logger.addHandler(logging.StreamHandler())

In [21]:
logger.info('hello world!')

hello world!
hello world!


In [29]:
logger.info("\033[31;47m你好IT网\033[0m")

你好IT网
你好IT网


In [4]:
def GetMonthList(from_date, to_date):
    from_dt = datetime.datetime.strptime(from_date, '%Y-%m-%d')
    to_dt = datetime.datetime.strptime(to_date, '%Y-%m-%d')
    month_plus1 = lambda dt:(dt.month == 12 and dt.replace(year = dt.year + 1, month = 1) or dt.replace(month = dt.month + 1))
    month_list = []
    current_dt = from_dt
    next_month_dt = month_plus1(current_dt)
    while current_dt <= to_dt:
        month_list.append((datetime.datetime.strftime(current_dt, '%Y-%m-%d'), 
                           datetime.datetime.strftime(min(next_month_dt-datetime.timedelta(days = 1) , to_dt), '%Y-%m-%d')))
        current_dt = next_month_dt
        next_month_dt = month_plus1(current_dt)
    return month_list
    

In [101]:
dt = datetime.datetime.strptime('2018-03-22', '%Y-%m-%d')
print(dt)
dt = dt.replace(month = dt.month + 1)
print(datetime.datetime.strftime(dt, '%Y-%m-%d'))

2018-03-22 00:00:00
2018-04-22


In [5]:
month_list = GetMonthList('2017-07-01', '2018-07-16')

In [140]:
for from_date,to_date in month_list:
    print('Fetching headers from %s to %s'%(from_date, to_date))
    SaveNewsHeaders(from_date, to_date, 'headers.%s.%s.json'%(from_date, to_date))

Fetching headers from 2018-05-01 to 2018-05-31
current progress: 1/44, header length: 0
current progress: 2/44, header length: 100
current progress: 3/44, header length: 200
current progress: 4/44, header length: 300
current progress: 5/44, header length: 400
current progress: 6/44, header length: 500
current progress: 7/44, header length: 600
current progress: 8/44, header length: 700
current progress: 9/44, header length: 800
current progress: 10/44, header length: 900
current progress: 11/44, header length: 1000
current progress: 12/44, header length: 1100
current progress: 13/44, header length: 1200
current progress: 14/44, header length: 1300
current progress: 15/44, header length: 1400
current progress: 16/44, header length: 1500
current progress: 17/44, header length: 1600
current progress: 18/44, header length: 1700
current progress: 19/44, header length: 1800
current progress: 20/44, header length: 1900
current progress: 21/44, header length: 2000
current progress: 22/44, head

In [6]:
def CombiningJsons(month_list, output_json_name):
    headers = []
    for from_date, to_date in month_list:
        with open('headers.%s.%s.json'%(from_date, to_date), 'rt') as f:
            headers += json.loads(f.read())
    with open(output_json_name, 'wt') as f:
        f.write(json.dumps(headers))

In [7]:
CombiningJsons(month_list, 'headers.2017-07-01.2018-07-16.json')

### Save headers into db

In [63]:
with open('headers.2017-07-01.2018-07-16.json', 'rt') as f:
    data = json.loads(f.read())
data0 = data[0]
for key,value in data0.items():
    print(key, value)




urlToImage https://assets.bwbx.io/images/users/iqjWHBFdfxIU/iIzxPSeCd490/v0/1200x800.jpg
source {'id': 'bloomberg', 'name': 'Bloomberg'}
author Thomas Penny
title U.K.'s May Targets Young Voters as She Looks to Silence Plotters
description U.K. Prime Minister Theresa May will seek to take back control of the U.K. political agenda and assert her authority over her Conservative Party at the start of its annual conference in Manchester on Sunday.
publishedAt 2017-09-30T23:30:00Z
url https://www.bloomberg.com/news/articles/2017-09-30/u-k-s-may-targets-young-voters-as-she-looks-to-silence-plotters


In [16]:
# Check if all headers include 'urlToImage', 'source', 'author', 'title', 'description', 'publishedAt', 'url'
attributes = ['urlToImage', 'source', 'author', 'title', 'description', 'publishedAt', 'url']
i = 0
for header in data:
    num_absence = 0
    num_attr = 0
    for attribute in attributes:
        num_attr += 1
        if attribute not in header:
            num_absence += 1
    if num_absence != 0 or num_attr != 7:
        i += 1
        print(header)
print(i)

0


In [58]:
conn.close()

In [14]:
conn = pymysql.connect(host = 'localhost',
                       db = 'FinNews',
                       port = 3306,
                       user = 'root',
                       password = '123456',
                       use_unicode = True,
                       charset = 'utf8')

In [15]:
cur = pymysql.cursors.Cursor(connection = conn)

In [44]:
cur.execute("""create table headers(
             id int auto_increment primary key,
             title varchar(1000),
             author text,
             published_at datetime,
             url text,
             urltoimage text,
             description text)
             """)
cur.execute("""ALTER TABLE headers ADD INDEX index_title(title)""")
conn.commit()


In [64]:
data = [(header['title'], header['author'], header['publishedAt'].replace('T', ' ').replace('Z',''), header['url'],
         header['urlToImage'], header['description']) for header in data]

In [54]:
for header in data:
    if (header[4] != None) and (type(header[4]) != str):
        print(len(header[4]))

In [65]:
cur.executemany("""insert into headers values(null, %s,%s,%s,%s,%s,%s)""", data)
conn.commit()

### Create table for articles

In [16]:
cur.execute("""
create table articles(
id int primary key,
article text)""")
conn.commit()

In [48]:
data[0]

("U.K.'s May Targets Young Voters as She Looks to Silence Plotters",
 'Thomas Penny',
 '2017-09-30T23:30:00Z',
 'https://www.bloomberg.com/news/articles/2017-09-30/u-k-s-may-targets-young-voters-as-she-looks-to-silence-plotters',
 'https://assets.bwbx.io/images/users/iqjWHBFdfxIU/iIzxPSeCd490/v0/1200x800.jpg',
 'U.K. Prime Minister Theresa May will seek to take back control of the U.K. political agenda and assert her authority over her Conservative Party at the start of its annual conference in Manchester on Sunday.')

In [78]:
df = pd.DataFrame(data)

In [81]:
gb = df.groupby(0).count()

In [82]:
gb[gb[5]>=2]

,1,2,3,4,5
0,,,,,
'Bloomberg Daybreak: Asia' Full Show,3,4,4,3,3
"A Single Parking Space in Hong Kong Has Sold for $765,000",3,3,3,3,3
A top analyst explains why everyone's overreacting to Facebook's data scandal (FB),2,2,2,2,2
Activists Aim at Defense Contractors Over Child Detention Policy,2,2,2,2,2
Africa's Richest Woman Prepares New Deals After Losing Oil Post,2,2,2,2,2
Ahold Delhaize CEO Unfazed by Increased Online Retail Rivalry,2,2,2,2,2
Airbnb Defeats Aimco Lawsuit Over Unauthorized Subleases,2,2,2,2,2
Akamai Is Working With Morgan Stanley on Strategic Review,1,2,2,2,2
Amazon Is Getting Into Sportswear,1,2,2,2,2


In [83]:
data = cur.execute('select id,title,url from headers where article_downloaded is NULL')

In [85]:
data = cur.fetchall()

In [86]:
len(data)

42290

In [4]:
que = queue.Queue()
que.put('123')

In [8]:
que.get_nowait()

Empty: 

In [9]:
que.empty()

True

In [12]:
if not que.empty():
    print('no empty')
else:
    print('empty')

SyntaxError: invalid syntax (<ipython-input-12-b236fe3f6900>, line 1)

### Counting word frequency of each article

In [125]:
conn = pymysql.connect(host = 'localhost',
                       db = 'FinNews',
                       port = 3306,
                       user = 'root',
                       password = '123456',
                       use_unicode = True,
                       charset = 'utf8')

In [126]:
cur = pymysql.cursors.Cursor(conn)

In [4]:
cur.execute(
"""create table wordcount(
ID int primary key,
counts text)""")
conn.commit()

In [12]:
def handleArticleBloomberg(text):
    return text[476:]
def ArticlePreprocess(articles):
    articles = list(map(handleArticleBloomberg, articles))
    articles = list(map(lambda x:list(gensim.utils.tokenize(x)), articles))
    return articles
def DictionaryGenerateModel(articles):
    # Train dictionary
    dct = gensim.corpora.Dictionary(articles)
    return dct

In [98]:
def WordCount(article):
    count = dict()
    for word in article:
        if word in count:
            count[word] += 1
        else:
            count[word] = 1
    return count

def CombineWordCount(articles):
    total_count = dict()
    for article in articles:
        for word in article:
            if word in total_count:
                total_count[word] += 1
            else:
                total_count[word] = 1
    return total_count
        

def InsertWordCount(conn):
    sql = """select ID,article from articles where ID not in (select * from(select ID from wordcount as tbl01) as tbl02)"""
    cur = pymysql.cursors.Cursor(conn)
    cur.execute(sql)
    raw_articles = cur.fetchall()
    articles = list(map(lambda x:x[1], raw_articles))
    IDs = list(map(lambda x:x[0], raw_articles))
    articles = handleArticleBloomberg(articles)
    articles = ArticlePreprocess(articles)
    print("Number of articles:", len(articles))
    for i,article in enumerate(articles):
        if i % 100 == 0:
            print(i)
            conn.commit()
        text = json.dumps(WordCount(article))
        text = text[:65000]
        cur.execute("""insert into wordcount values(%d, %s)""", (IDs[i], text))
    conn.commit()
    cur.close()

In [65]:
InsertWordCount(conn)

Number of articles: 0


### Count periodic word frequency

In [127]:
cur.execute("""
create table wordcount_daily(
day datetime primary key,
counts mediumtext)""")
conn.commit()

In [69]:
cur.execute("""select tbl01.ID,tbl01.published_at,tbl02.counts from wordcount tbl02 left join headers tbl01 on (tbl01.ID = tbl02.ID) order by tbl01.published_at""")
counts = cur.fetchall()

In [134]:
time_begin = counts[0][1]
time_end   = counts[-1][1]
time_begin = time_begin.replace(hour = 0, minute = 0, second = 0)
time_end = (time_end + pd.Timedelta(days = 1)).replace(hour = 0, minute = 0, second = 0)
drange = pd.date_range(start = time_begin, end = time_end)

i = 0
for date in drange:
    articles = []
    while (i < len(counts)) and (pd.Timestamp(counts[i][1]) < pd.Timestamp(date)) :
        articles.append(json.loads(counts[i][2]))
        i += 1
    day_count = CombineWordCount(articles)
    day = pd.Timestamp.strftime(pd.Timestamp(date) - pd.Timedelta(days = 1), '%Y-%m-%d')
    text = json.dumps(day_count)
    print(len(text))
    cur.execute("""replace into wordcount_daily(day,counts) values('%s', '%s')"""%(day, text))
    conn.commit()

2
106946
165761
160866
135568
53409
90487
157141
190716
163713
163211
127961
71775
85421
177189
184661
176585
169957
155216
58644
87376
167128
169037
171125
149598
141082
72402
80829
169713
183747
170435
179288
157704
59064
77394
177135
170412
169976
189419
141649
48331
85913
178416
166607
161587
173626
150614
42405
60186
141314
150002
170053
191175
163760
38442
78642
171960
169183
172387
176319
140777
50248
68377
173091
178653
173011
167275
156760
40127
71177
177241
171427
164123
103981
149070
35709
55663
163131
161522
185354
172826
150389
46443
66776
160674
168662
171563
162685
166612
29910
65072
168555
153702
163548
181351
150990
42257
66826
168518
164210
167123
157897
122126
31072
27661
34232
95469
112257
136144
89506
19553
34677
49662
137752
146964
154388
137724
28473
52437
148164
172816
183800
148729
156185
44509
65691
131295
166976
165941
171054
151174
46404
63614
164417
166431
186836
179728
143723
34124
53693
157054
161781
183582
168163
159489
21159
62155
177978
166652
184594
1

In [209]:
cur.execute("""
create table wordcount_weekly(
day datetime primary key,
counts mediumtext)""")
conn.commit()

In [210]:
time_begin = counts[0][1]
time_end   = counts[-1][1]

In [211]:
drange = pd.date_range(start = time_begin, end = time_end)
drange = list(drange[0::7])
drange.append(drange[-1] + pd.Timedelta(days = 7))

In [212]:
i = 0
for date in drange:
    articles = []
    while (i < len(counts)) and (pd.Timestamp(counts[i][1]) < pd.Timestamp(date)) :
        articles.append(json.loads(counts[i][2]))
        i += 1
    day_count = CombineWordCount(articles)
    day = pd.Timestamp.strftime(pd.Timestamp(date) - pd.Timedelta(days = 7), '%Y-%m-%d')
    text = json.dumps(day_count)
    print(len(text))
    cur.execute("""replace into wordcount_weekly(day,counts) values('%s', '%s')"""%(day, text))
    conn.commit()

2
397793
400730
408049
391406
416441
409545
380936
420083
392300
399038
362285
408292
386751
397978
309615
269681
367104
382998
391719
386813
413974
405446
379324
388240
411007
398051
420859
393825
335488
377889
399193
390074
413225
400052
393077
387316
383298
333006
400806
375800
374048
317605
308886
278539


In [202]:
cur.execute("""
create table wordcount_monthly(
day datetime primary key,
counts mediumtext)""")
conn.commit()

In [203]:
def AddOneMonth(date):
    if date.month == 12:
        date = date.replace(year = date.year + 1, month = 1)
    else:
        date = date.replace(month = date.month + 1)
    return date

def SubOneMonth(date):
    if date.month == 1:
        date = date.replace(year = date.year - 1, month = 12)
    else:
        date = date.replace(month = date.month - 1)
    return date

In [204]:
time_begin = counts[0][1]
time_end   = counts[-1][1]
time_begin = time_begin.replace(day = 1, hour = 0, minute = 0, second = 0)
time_end = AddOneMonth(time_end).replace(day = 1, hour = 0, minute = 0, second = 0)

In [205]:
def GenerateDataRangeMonthly(time_begin, time_end):
    time_begin = time_begin.replace(day = 1, hour = 0, minute = 0, second = 0)
    date = time_begin
    drange = [date]
    while date < time_end:
        date = AddOneMonth(date)
        drange.append(date)
    return list(map(lambda x:pd.Timestamp.strftime(x, '%Y-%m-%d'), drange))

In [206]:
time_begin = counts[0][1]
time_end   = counts[-1][1]
time_begin = time_begin.replace(day = 1, hour = 0, minute = 0, second = 0)
time_end = AddOneMonth(time_end).replace(day = 1, hour = 0, minute = 0, second = 0)
drange = GenerateDataRangeMonthly(time_begin, time_end)

i = 0
for date in drange:
    articles = []
    while (i < len(counts)) and (pd.Timestamp(counts[i][1]) < pd.Timestamp(date)) :
        articles.append(json.loads(counts[i][2]))
        i += 1
    day_count = CombineWordCount(articles)
    day = pd.Timestamp.strftime(SubOneMonth(pd.Timestamp(date)), '%Y-%m-%d')
    text = json.dumps(day_count)
    print(len(text))
    cur.execute("""replace into wordcount_monthly(day,counts) values('%s', '%s')"""%(day, text))
    conn.commit()

2
644012
821716
806632
723462
788208
776717
810105
790955
810111
762010
416530


In [63]:
sql = """select ID,article from articles where ID not in (select * from(select ID from wordcount as tbl01) as tbl02)"""
cur.execute(sql)
raw_articles = cur.fetchall()
articles = list(map(lambda x:x[1], raw_articles))
IDs = list(map(lambda x:x[0], raw_articles))
articles = handleArticleBloomberg(articles)
articles = ArticlePreprocess(articles)
print("Number of articles:", len(articles))
for i,article in enumerate(articles):
    if i % 100 == 0:
        print(i)
        conn.commit()
    cur.execute("""insert into wordcount values(%d, %s)""", (IDs[i], json.dumps(WordCount(article))))

Number of articles: 0


In [47]:
articles[0]

['Senate',
 'Republicans',
 'Give',
 'Up',
 'Obamacare',
 'Repeal',
 'BidMajority',
 'Leader',
 'Mitch',
 'McConnell',
 'started',
 'the',
 'year',
 'with',
 'a',
 'reputation',
 'as',
 'a',
 'master',
 'Senate',
 'puppeteer',
 'These',
 'days',
 'he',
 's',
 'looking',
 'more',
 'like',
 'the',
 'puppet',
 'McConnell',
 'was',
 'mostly',
 'a',
 'bystander',
 'on',
 'Tuesday',
 'as',
 'events',
 'in',
 'the',
 'Senate',
 'went',
 'from',
 'bad',
 'to',
 'worse',
 'Another',
 'Obamacare',
 'repeal',
 'effort',
 'collapsed',
 'A',
 'hard',
 'right',
 'Alabama',
 'conservative',
 'who',
 'vilifies',
 'him',
 'by',
 'name',
 'won',
 'a',
 'Senate',
 'GOP',
 'primary',
 'And',
 'one',
 'of',
 'his',
 'closest',
 'allies',
 'in',
 'the',
 'chamber',
 'Bob',
 'Corker',
 'of',
 'Tennessee',
 'said',
 'he',
 'will',
 'retire',
 'after',
 'next',
 'year',
 'Both',
 'Corker',
 's',
 'retirement',
 'and',
 'Roy',
 'Moore',
 's',
 'victory',
 'over',
 'McConnell',
 'backed',
 'Senator',
 'Luther',
